In [1]:
import sys
import pandas as pd
import numpy as np
from collections import defaultdict
from scipy.optimize import minimize
from csv import DictReader

from glicko import run_league, read_csv

See [NathanDeMaria/EndGame/ncaa_fb.R](https://github.com/NathanDeMaria/EndGame/blob/f1b61777e1550575ed4d23d1ee2920ecf1bb01dd/ncaa_fb.R) for the `.csv`s

In [2]:
conferences_path = './ncaaf_conferences.csv'
games_path = './ncaaf.csv'

In [3]:
fbs_conferences = {
    # Power 5
    1, 4, 5, 8, 9,
    # Group of 5
    12, 15, 151, 17, 37, 
    # Indep
    18,
}

conferences = pd.read_csv(conferences_path)
fbs_lookup = {
    row.team: (row.team_conf in fbs_conferences)
    for _, row in conferences.iterrows()
}
def is_fbs(team_name: str) -> bool:
    return fbs_lookup[team_name]

In [4]:
league = read_csv(games_path)

In [5]:
def create_offseason_runner(
    init_variance: float,
    variance_over_time: float,
    fbs_advantage: float,
):
    def run_offseason(league, season: int) -> None:
        for team in league.teams:
            try:
                mean, var = team.rating
                team.update_rating(
                    (mean, var + variance_over_time),
                    season,
                    0
                )
            except ValueError:
                # i.e. this was the first season.
                multiplier = .5 if is_fbs(team.name) else -.5
                team.update_rating((1500 + fbs_advantage * multiplier, init_variance), season, 0)
    return run_offseason

In [6]:
def evaluate(x):
    runner = create_offseason_runner(*x)
    
    # reset hack
    for team in league.teams:
        team._ratings = dict()
        team.games = defaultdict(list)
    
    return run_league(league, runner)[0]

In [7]:
init = [132653.8137034986, 23135.41931163794, 557.4684194903585]

In [8]:
result = minimize(
    evaluate,
    x0=np.array(init),
    options=dict(disp=True),
    bounds=((0, None), (0, None), (0, None))
)

init = list(result.x)

In [8]:
print(init)

[132653.8137034986, 23135.41931163794, 557.4684194903585]


In [9]:
runner = create_offseason_runner(*init)

# reset hack
for team in league.teams:
    team._ratings = dict()
    team.games = defaultdict(list)

discrepancy, teams = run_league(league, runner)

In [10]:
# Top teams
for i, t in enumerate(sorted(teams, key=lambda x: -x.rating[0])):
    games = t.games[2018]
    wins = len([g for g in games if g.score > .5])
    losses = len(games) - wins
    print(i + 1, f'({wins}-{losses})', t.name, t.rating)

1 (8-0) Alabama Crimson Tide (2242.8505807676306, 20594.69328795934)
2 (8-0) Clemson Tigers (2171.573549322579, 16072.312830275216)
3 (7-1) Georgia Bulldogs (2116.2998579694213, 14549.762329036845)
4 (7-1) LSU Tigers (2085.8124447871855, 12788.612820247567)
5 (7-1) Oklahoma Sooners (2084.3731607066325, 13597.264106143568)
6 (7-1) Michigan Wolverines (2078.2826819209313, 12789.75769162057)
7 (7-1) Ohio State Buckeyes (2072.717871825869, 15359.487988677549)
8 (6-2) Washington Huskies (2071.8965236241356, 13644.622767022234)
9 (6-2) Penn State Nittany Lions (2055.4890541961017, 14166.993911845217)
10 (7-0) Notre Dame Fighting Irish (2054.2164658259, 13765.849728773383)
11 (6-2) Iowa Hawkeyes (2044.7808807727954, 12435.64659537782)
12 (6-2) Utah Utes (2014.5611182796902, 12530.968894021908)
13 (5-2) Stanford Cardinal (2011.297135816114, 13457.875907103225)
14 (5-3) Auburn Tigers (2009.2685634534018, 14336.928799736901)
15 (5-3) Wisconsin Badgers (1999.9059697944026, 15493.085754893555)
16 